In [25]:
import json
import pandas as pd
import numpy as np
import random
import string

In [31]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

## Load Data

In [94]:
filename = "dataset/web_science_dataset.jsonl"
json_data = []
with open(filename) as f:
    json_data = f.readlines()
json_data_list = []
for item in json_data:
    json_data_list.append(json.loads(item))
df = pd.DataFrame(json_data_list)

#labels
label_map = dict(zip(df['category'],df['categoryId']))
label_map

{'medical-science': 2,
 'nutrition': 0,
 'psychology': 4,
 'climate-change': 1,
 'physics': 3}

#### Preprocessing the dataset

In [126]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return (' '.join(lemmatizer.lemmatize(w) for w in word_tokenize(text)))

In [127]:
def preprocess_text(df):    

    # remove leading/trailing spaces
    df = df.str.strip()
    
    # convert to lowercase
    df = df.str.lower()
    
    #df = df.replace(to_replace ='http\S+', value = '', regex = True)
    
    # remove punctuation
    translator = str.maketrans('', '', string.punctuation) 
    df = df.str.translate(translator)
    
    # remove non-alphanumeric characters
    df = df.replace(to_replace ='\s*[^A-Za-z0-9]+\s*', value = ' ', regex = True)
    
    # remove digits
    translator = str.maketrans('', '', string.digits) 
    df = df.str.translate(translator)
    
    df = df.str.strip()
    
    #lemmatize
    df = df.apply(lemmatize_text)
    
    return df

In [97]:
df['question'] = preprocess_text(df['question'])
df.head()

,question,questionId,questionUrl,category,categoryId,answer,answerUrl,answerId
0,can headbanging cause brain damage,14138,https://skeptics.stackexchange.com/questions/1...,medical-science,2,A number of injuries have been attributed to t...,https://skeptics.stackexchange.com/questions/1...,14139
1,doe the shangrila diet work according to it su...,10103,https://skeptics.stackexchange.com/questions/1...,nutrition,0,The Shangri-La diet depends on two theories:\n...,https://skeptics.stackexchange.com/questions/1...,16121
2,can phobia be genetic but created in one gener...,18713,https://skeptics.stackexchange.com/questions/1...,psychology,4,This question has remained unanswered yet not ...,https://skeptics.stackexchange.com/questions/1...,22322
3,do of u american think that global warming is ...,36010,https://skeptics.stackexchange.com/questions/3...,climate-change,1,The&nbsp;40% figure most likely comes from Pew...,https://skeptics.stackexchange.com/questions/3...,36011
4,doe boiling the same water twice make it dange...,11118,https://skeptics.stackexchange.com/questions/1...,nutrition,0,The claims\n\n\nevery time the same water is b...,https://skeptics.stackexchange.com/questions/1...,11119


#### Stratified splits

In [98]:
from sklearn.model_selection import train_test_split

In [99]:
train_data, test_data, train_labels, test_labels = train_test_split(
        df, df['category'],stratify=df['category'], test_size=0.4)

test_data, valid_data, test_labels, valid_labels = train_test_split(
        test_data, test_labels, stratify=test_labels, test_size=0.5)

In [100]:
train_data.head()

,question,questionId,questionUrl,category,categoryId,answer,answerUrl,answerId
120,do alcoholic digestive help digestion,7410,https://skeptics.stackexchange.com/questions/7410,nutrition,0,"Maybe, maybe not...\n\nReference - Effect on g...",https://skeptics.stackexchange.com/questions/7...,7420
627,is sodium nitrite food additive e carcinogenic,28106,https://skeptics.stackexchange.com/questions/2...,nutrition,0,"There's good evidence for it.\n\n""Red and proc...",https://skeptics.stackexchange.com/questions/2...,28416
28,doe eating sugar feed yeast in the gut,18987,https://skeptics.stackexchange.com/questions/1...,nutrition,0,Auto-brewery syndrome is a well-known but rare...,https://skeptics.stackexchange.com/questions/1...,18988
893,can you deduce the quality of watermelon by ta...,20567,https://skeptics.stackexchange.com/questions/2...,nutrition,0,There are a few papers assessing methods and r...,https://skeptics.stackexchange.com/questions/2...,20586
1014,doe a lack of radiation shadow disprove the bi...,34226,https://skeptics.stackexchange.com/questions/3...,physics,3,The Big Bang was the whole universe. There was...,https://skeptics.stackexchange.com/questions/3...,34231


## Model

In [101]:
# Parts of code provided in the tutorial was used. Reference: liar_liar_bilstm.ipynb

In [102]:
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook as tqdm
from torch import nn
from torch.optim import Adam
from typing import List, Tuple

In [103]:
## defining global parameters

In [104]:
seed = 1000
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

num_labels = len(label_map)
batch_size = 64
lr = 2e-4
lstm_dim = 200
n_epochs = 20
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")


#### Sub Word Byte Pair word embeddings

In [105]:
from bpemb import BPEmb

bpemb_en = BPEmb(lang='en', dim=300, vs=25000)
pretrained_embeddings = np.concatenate([bpemb_en.emb.vectors, np.zeros(shape=(1,300))], axis=0)
vocabulary = bpemb_en.emb.index2word + ['[PAD]']

### Loading data into the model

In [106]:
class ClassificationDatasetReader():
    
    def __init__(self,df,tokenizer):
        self.df = df
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        row = self.df.values[idx]
        input_seq,seq_lens = text_to_batch_bilstm([row[0]], self.tokenizer) #sending a list of one row
        label = label_map[row[3]]
        return input_seq, seq_lens, label

In [107]:
def text_to_batch_bilstm(text: List, tokenizer) -> Tuple[List, List]:
    
    """tokenize and return tokenized ids and their lengths"""
    input_ids = [tokenizer.encode_ids_with_eos(t) for t in text]
    return input_ids, [len(ids) for ids in input_ids] #input_ids is list of lists [[token_ids_for_row1],[token_ids_row2]]

In [108]:
def collate_batch_bilstm(input_data: Tuple) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    input_ids = [i[0][0] for i in input_data]
    seq_lens = [i[1][0] for i in input_data]
    labels = [i[2] for i in input_data]

    max_length = max([len(i) for i in input_ids])

    input_ids = [(i + [25000] * (max_length - len(i))) for i in input_ids]

    assert (all(len(i) == max_length for i in input_ids))
    return torch.tensor(input_ids), torch.tensor(seq_lens), torch.tensor(labels)

In [109]:
# Create the dataset readers and read data
train_dataset = ClassificationDatasetReader(train_data, bpemb_en)
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch_bilstm)

valid_dataset = ClassificationDatasetReader(valid_data, bpemb_en)
valid_dl = DataLoader(valid_dataset, batch_size=len(valid_data), collate_fn=collate_batch_bilstm)

### Creating the model

In [130]:
# Define the model
class LSTMNetwork(nn.Module):

    def __init__(self, pretrained_embeddings: torch.tensor, lstm_dim: int, dropout_prob: float = 0.1, n_classes: int = 5):
        
        super(LSTMNetwork, self).__init__()
        
        #LSTM input
        self.input_size = pretrained_embeddings.shape[1] #features in input
        self.hidden_size = 200 
        self.num_layers = 1 #default 
        
        self.embeddings = nn.Embedding.from_pretrained(pretrained_embeddings, padding_idx=pretrained_embeddings.shape[0] - 1)
        self.lstm = nn.LSTM(self.input_size,self.hidden_size,self.num_layers,batch_first=True,dropout=dropout_prob,bidirectional=True)
        self.ff = nn.Linear(2*self.hidden_size,n_classes)
        self.n_classes = n_classes
        
    def forward(self, inputs, input_lens, labels = None):
        
        # Get embeddings (b x sl x edim)
        embeds = self.embeddings(inputs)
        lstm_out, hidden = self.lstm(embeds)
        
        # Get the last output for classification (b x 2*lstm_dim)
        ff_in = lstm_out.gather(1, input_lens.view(-1,1,1).expand(lstm_out.size(0), 1, lstm_out.size(2)) - 1).squeeze()
        #print(input_lens.view(-1,1,1).expand(lstm_out.size(0), 1, lstm_out.size(2)))
        
        # Get logits (b x 2)
        logits = self.ff(ff_in).view(-1, self.n_classes)
        
        outputs = (logits,)
        if labels is not None:
            # Xentropy loss
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
            outputs = (loss,) + outputs

        return outputs
      

In [131]:
# Create the model
model = LSTMNetwork(
    pretrained_embeddings=torch.FloatTensor(pretrained_embeddings), 
    lstm_dim=lstm_dim, 
    dropout_prob=0, 
    n_classes=len(label_map)
  ).to(device)

In [132]:
def accuracy(logits, labels):
    return torch.sum(torch.argmax(logits, dim=-1) == labels).type(torch.float) / float(labels.shape[0])


In [135]:
def train(model, train_dl, valid_dl, optimizer, n_epochs, device):
    
    losses = []
    best_acc = 0.0
  
    for ep in range(n_epochs):
        loss_epoch = []
        
        for batch in tqdm(train_dl):
            
            model.train()
            optimizer.zero_grad()
            
            batch = tuple(t.to(device) for t in batch)
            input_ids = batch[0]
            seq_lens = batch[1]
            labels = batch[2]

            loss, logits = model(input_ids, seq_lens, labels=labels)
            losses.append(loss.item())
            loss_epoch.append(loss.item())
      
            #print(loss.dtype)
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
        #gc.collect()
    
        # Evaluate
        model.eval()
        with torch.no_grad():
            for batch in valid_dl:

                batch = tuple(t.to(device) for t in batch)
                input_ids = batch[0]
                seq_lens = batch[1]
                labels = batch[2]

                val_loss, logits = model(input_ids, seq_lens, labels=labels)
                acc = accuracy(logits, labels)

                print(f'Validation accuracy: {acc}, train loss: {sum(loss_epoch) / len(loss_epoch)}',"val_loss: ",val_loss.item())
                best_model = model.state_dict()

                if acc > best_acc:
                  #best_model = model.state_dict()
                    best_acc = acc
                #gc.collect()

    model.load_state_dict(best_model)
    return model, losses

In [134]:
# Create the optimizer
optimizer = Adam(model.parameters(), lr=lr)

# Train
model, losses = train(model, train_dl, valid_dl, optimizer, n_epochs, device)

tensor(1.6006, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5996, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5869, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5834, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5753, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5700, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5818, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5722, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5480, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5481, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32

tensor(1.5469, device='cuda:0')
Validation accuracy: 0.37383174896240234, train loss: 1.5766063809394837 val_loss:  1.5469024181365967


tensor(1.5694, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5074, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5380, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5527, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4867, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5207, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4936, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5016, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5307, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5961, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32

tensor(1.5009, device='cuda:0')
Validation accuracy: 0.37383174896240234, train loss: 1.5296932816505433 val_loss:  1.5008717775344849


tensor(1.5818, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4874, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4468, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4916, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4620, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5120, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5153, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.3840, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4657, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4988, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32

tensor(1.4614, device='cuda:0')
Validation accuracy: 0.37383174896240234, train loss: 1.4845430493354796 val_loss:  1.4614430665969849


tensor(1.4454, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4238, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4752, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4042, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4743, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4093, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4183, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4790, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5280, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4162, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32

tensor(1.4142, device='cuda:0')
Validation accuracy: 0.37383174896240234, train loss: 1.447383451461792 val_loss:  1.4142210483551025


tensor(1.3209, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.3586, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.3576, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.5504, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4271, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4034, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.4057, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.3501, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.3504, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(1.3705, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32

tensor(1.3209, device='cuda:0')
Validation accuracy: 0.37383174896240234, train loss: 1.3894691109657287 val_loss:  1.3208885192871094


...

tensor(0.5616, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5705, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4416, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5054, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.7157, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4944, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5499, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5837, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5438, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5435, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32

tensor(0.6814, device='cuda:0')
Validation accuracy: 0.7710280418395996, train loss: 0.5510117322206497 val_loss:  0.6813513040542603


tensor(0.6149, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.6729, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4080, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4401, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5193, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4927, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5042, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4639, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5351, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4604, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32

tensor(0.6791, device='cuda:0')
Validation accuracy: 0.7616822123527527, train loss: 0.5111506819725037 val_loss:  0.6790711879730225


tensor(0.4888, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5244, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4767, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4853, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4946, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5062, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5159, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.3746, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4436, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4165, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32

tensor(0.6717, device='cuda:0')
Validation accuracy: 0.7757009267807007, train loss: 0.4726714134216309 val_loss:  0.6716684103012085


tensor(0.5209, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.3640, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4556, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.5114, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4483, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4698, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4227, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.3572, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.3650, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4420, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32

tensor(0.6932, device='cuda:0')
Validation accuracy: 0.7570093274116516, train loss: 0.43568646907806396 val_loss:  0.6931828260421753


tensor(0.4520, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4917, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.3970, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.3669, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.2825, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4980, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.3157, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4088, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.3476, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32
tensor(0.4402, device='cuda:0', grad_fn=<NllLossBackward>)
torch.float32

tensor(0.6717, device='cuda:0')
Validation accuracy: 0.7710280418395996, train loss: 0.4000479310750961 val_loss:  0.671684205532074
